In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 17:51:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/03 17:51:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet('../data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/home/ndirangu749/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS hourly_amount,
    COUNT(1) AS hourly_number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+---------------------+
|               hour|zone|     hourly_amount|hourly_number_records|
+-------------------+----+------------------+---------------------+
|2020-01-28 19:00:00| 134|193.61000000000007|                   17|
|2020-01-22 19:00:00|  65| 657.0300000000001|                   41|
|2020-01-27 08:00:00|  17|             85.56|                    4|
|2020-01-02 09:00:00|  66|229.39999999999998|                   12|
|2020-01-02 12:00:00|  89|310.28000000000003|                   14|
|2020-01-07 12:00:00|  66|             179.5|                    9|
|2020-01-03 08:00:00| 223|165.90000000000003|                    9|
|2020-01-17 10:00:00|  41| 638.2699999999999|                   49|
|2020-01-02 11:00:00|  26|             198.6|                    7|
|2020-01-12 20:00:00| 247|36.900000000000006|                    3|
|2020-01-12 15:00:00|  10|             75.92|                    1|
|2020-01-24 04:00:00| 129|148.89000000000001|   

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('../data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS hourly_amount,
    COUNT(1) AS hourly_number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue.show()

+-------------------+----+------------------+---------------------+
|               hour|zone|     hourly_amount|hourly_number_records|
+-------------------+----+------------------+---------------------+
|2020-01-10 06:00:00| 144|            149.51|                    9|
|2020-01-29 03:00:00|  13|            248.89|                    6|
|2020-01-13 21:00:00| 255|            108.98|                    8|
|2020-01-04 12:00:00| 249| 2245.569999999999|                  152|
|2020-01-02 15:00:00| 236| 7382.910000000018|                  504|
|2020-01-31 21:00:00|  50|1784.4399999999994|                  104|
|2020-01-07 12:00:00|  68| 3046.229999999997|                  181|
|2020-01-02 09:00:00| 161|4299.9299999999985|                  242|
|2020-01-23 17:00:00| 233| 4642.769999999999|                  235|
|2020-01-06 22:00:00| 158| 923.2100000000003|                   57|
|2020-01-15 07:00:00|  13|2432.6299999999997|                  118|
|2020-01-23 19:00:00| 224|            327.28|   

In [11]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../data/report/revenue/yellow', mode='overwrite')

In [25]:
!tree ../data/report/

../data/report/
└── revenue
    ├── _SUCCESS
    ├── green
    │   ├── _SUCCESS
    │   ├── part-00000-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00001-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00002-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00003-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00004-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00005-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00006-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00007-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00008-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00009-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00010-2662b5fe-43af-4ce3-be3b-1fb9a4590532-c000.snappy.parquet
    │   ├── part-00011-2662b5fe-43af-4ce

In [12]:
df_green_revenue = spark.read.parquet('../data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../data/report/revenue/yellow')

In [13]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('hourly_amount', 'green_amount') \
    .withColumnRenamed('hourly_number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('hourly_amount', 'yellow_amount') \
    .withColumnRenamed('hourly_number_records', 'yellow_number_records')

In [14]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  25| 531.0000000000002|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  43|            107.52|                   6| 6539.510000000012|                  390|
|2020-01-01 00:00:00|  49|26

In [15]:
df_join.write.parquet('../data/report/revenue/total', mode='overwrite')

In [18]:
df_join = spark.read.parquet('../data/report/revenue/total')

In [19]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [21]:
df_zones = spark.read.parquet('zones/')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [23]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00| 

In [24]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00| 